In [4]:
import cv2
from numpy import random
import numpy as np

# Mean Square Error (MSE) calculation
def MSE(image_a, image_b):
    w, h, c = image_a.shape
    mse = np.sum((image_a - image_b) ** 2) / (w * h * c)
    return mse

# Self-Organizing Map (SOM) for Colour Palette Generation
def ColourPaletteGeneration(image_in, epochs=10, initial_learn_rate=0.1):
    w, h, c = image_in.shape
    codebook = random.rand(8, 8, 3) * 255  # Initialize codebook with random values scaled to [0, 255]

    learn_rate = initial_learn_rate
    for epoch in range(epochs):
        for i in range(w * h):
            # Randomly select a pixel
            randomPixelRow = random.randint(0, h)
            randomPixelCol = random.randint(0, w)
            currentPixel = image_in[randomPixelRow, randomPixelCol, :]

            # Find the winner codeword
            minDist = np.inf
            minIndex_r, minIndex_c = 0, 0
            for r in range(8):
                for c in range(8):
                    dist = np.linalg.norm(currentPixel - codebook[r, c, :])
                    if dist < minDist:
                        minDist = dist
                        minIndex_r, minIndex_c = r, c

            # Update the winner's weight
            codebook[minIndex_r, minIndex_c, :] += learn_rate * (currentPixel - codebook[minIndex_r, minIndex_c, :])

        # Decay learning rate
        learn_rate *= 0.99  # Reduce learning rate over time

    return codebook

# Map the image to the quantized palette
def PixelMapping(image_in, codebook):
    w, h, c = image_in.shape
    image_out = np.zeros((w, h, c), dtype=np.uint8)

    for r in range(h):
        for c in range(w):
            currentPixel = image_in[r, c, :]

            # Find the winner codeword
            minDist = np.inf
            minIndex_r, minIndex_c = 0, 0
            for i in range(8):
                for j in range(8):
                    dist = np.linalg.norm(currentPixel - codebook[i, j, :])
                    if dist < minDist:
                        minDist = dist
                        minIndex_r, minIndex_c = i, j

            image_out[r, c, :] = codebook[minIndex_r, minIndex_c, :]

    return image_out

# Example usage
if __name__ == "__main__":
    # Load the image
    image_a = cv2.imread('baboon.jpeg')

    # Generate the color palette using SOM
    palette = ColourPaletteGeneration(image_a, epochs=10, initial_learn_rate=0.1)

    # Quantize the image
    quantized_image = PixelMapping(image_a, palette)

    # Calculate MSE between the original and quantized images
    mse_value = MSE(image_a, quantized_image)
    print(f'MSE: {mse_value}')

    # Save the quantized image
    cv2.imwrite('Quantized_Baboon.jpeg', quantized_image)


MSE: 59.39032098765432
